# Output Parsers

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

- `get_format_instructions() -> str`: A method which returns a string containing instructions for how the output of a language model should be formatted.
- `parse(str) -> Any`: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

Below we go over some examples of output parsers.

## Structured Output Parser

This output parser can be used when you want to return multiple fields.

In [2]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

Here we define the response schema we want to receive.

In [21]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

We now get a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.

In [22]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [9]:
print(prompt.format_prompt(question="what's the capital of france").text)

answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema:

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```
what's the capital of france


We can now use this to format a prompt to send to the language model, and then parse the returned result.

In [10]:
model = OpenAI(temperature=0)

In [11]:
_input = prompt.format_prompt(question="what's the capital of france")
output = model(_input.to_string())

In [12]:
print(output)

?

```json
{
	"answer": "Paris",
	"source": "https://en.wikipedia.org/wiki/Paris"
}
```


In [13]:
output_parser.parse(output)

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}

And here's an example of using this in a chat model

In [14]:
chat_model = ChatOpenAI(temperature=0)

In [23]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible.\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [28]:
# prompt._merge_partial_and_user_variables(question="what's the capital of france")
prompt.format_prompt(question="what's the capital of france")


KeyError: 'format_instructions'

In [17]:
_input = prompt.format_prompt(question="what's the capital of france")
output = chat_model(_input.to_messages())

KeyError: 'format_instructions'

In [11]:
output_parser.parse(output.content)

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}

## CommaSeparatedListOutputParser

This output parser can be used to get a list of items as output.

In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [13]:
output_parser = CommaSeparatedListOutputParser()

In [14]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [15]:
model = OpenAI(temperature=0)

In [16]:
_input = prompt.format(subject="ice cream flavors")
output = model(_input)

In [17]:
output_parser.parse(output)

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']